Objective3:WAP to implement a three-layer neural network using Tensor flow library (only, no keras) to classify MNIST handwritten digits dataset. Demonstrate the implementation of feed-forward and back-propagation approaches

In [ ]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

# Disable eager execution (for TensorFlow v2 compatibility with v1 placeholders)
tf.compat.v1.disable_eager_execution()

# Load MNIST dataset
mnist = tfds.load('mnist', split=['train', 'test'], as_supervised=True)
train_data, test_data = mnist

# Function to preprocess data
def preprocess(dataset):
    images, labels = [], []
    for img, label in tfds.as_numpy(dataset):
        images.append(img.flatten() / 255.0)  
        labels.append(np.eye(10)[label])  #one hot encoding
    return np.array(images), np.array(labels)

# Prepare train and test datasets
train_images, train_labels = preprocess(train_data)
test_images, test_labels = preprocess(test_data)

# Define network parameters
input_size = 784  
hidden_size1 = 128
hidden_size2 = 64
output_size = 10  
learning_rate = 0.01
epochs = 20
batch_size = 100

# Define placeholders for input and output
X = tf.compat.v1.placeholder(tf.float32, [None, input_size])
Y = tf.compat.v1.placeholder(tf.float32, [None, output_size])

# Initialize weights and biases
weights = {
    'w1': tf.Variable(tf.random.normal([input_size, hidden_size1])),
    'w2': tf.Variable(tf.random.normal([hidden_size1, hidden_size2])),
    'w3': tf.Variable(tf.random.normal([hidden_size2, output_size]))
}

biases = {
    'b1': tf.Variable(tf.random.normal([hidden_size1])),
    'b2': tf.Variable(tf.random.normal([hidden_size2])),
    'b3': tf.Variable(tf.random.normal([output_size]))
}

# Define the feed-forward network
def neural_network(x):
    layer1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])#matmul:matrix multiplication of weights and biases
    layer1 = tf.nn.sigmoid(layer1)  # Using Sigmoid activation function

    layer2 = tf.add(tf.matmul(layer1, weights['w2']), biases['b2'])
    layer2 = tf.nn.sigmoid(layer2)  

    output_layer = tf.add(tf.matmul(layer2, weights['w3']), biases['b3'])
    return output_layer

# Forward propagation
logits = neural_network(X)
prediction = tf.nn.softmax(logits)

# Define loss function and optimizer (Backpropagation)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(loss)

# model evaluation
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32)) * 100

#  model training
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())

    for epoch in range(epochs):
        total_batches = len(train_images) // batch_size
        avg_loss = 0

        for i in range(total_batches):
            batch_x = train_images[i * batch_size:(i + 1) * batch_size]
            batch_y = train_labels[i * batch_size:(i + 1) * batch_size]
            _, batch_loss = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y})
            avg_loss += batch_loss / total_batches

        acc = sess.run(accuracy, feed_dict={X: test_images, Y: test_labels})
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, Test Accuracy: {acc:.2f}%")  

    # Final test accuracy in percentage
    test_acc = sess.run(accuracy, feed_dict={X: test_images, Y: test_labels})
    print(f"\nFinal Test Accuracy: {test_acc:.2f}% ")


Epoch 1, Loss: 0.4955, Test Accuracy: 91.96%
Epoch 2, Loss: 0.1943, Test Accuracy: 94.18%
Epoch 3, Loss: 0.1435, Test Accuracy: 95.08%
Epoch 4, Loss: 0.1125, Test Accuracy: 95.45%
Epoch 5, Loss: 0.1003, Test Accuracy: 95.51%
Epoch 6, Loss: 0.0866, Test Accuracy: 95.78%
Epoch 7, Loss: 0.0780, Test Accuracy: 95.97%
Epoch 8, Loss: 0.0727, Test Accuracy: 95.86%
Epoch 9, Loss: 0.0635, Test Accuracy: 96.09%
Epoch 10, Loss: 0.0664, Test Accuracy: 96.27%
Epoch 11, Loss: 0.0589, Test Accuracy: 96.27%
Epoch 12, Loss: 0.0583, Test Accuracy: 96.08%
Epoch 13, Loss: 0.0550, Test Accuracy: 96.48%
Epoch 14, Loss: 0.0537, Test Accuracy: 96.43%
Epoch 15, Loss: 0.0461, Test Accuracy: 96.15%
Epoch 16, Loss: 0.0458, Test Accuracy: 96.36%
Epoch 17, Loss: 0.0454, Test Accuracy: 96.34%
Epoch 18, Loss: 0.0459, Test Accuracy: 96.29%
Epoch 19, Loss: 0.0426, Test Accuracy: 96.90%
Epoch 20, Loss: 0.0400, Test Accuracy: 96.32%

Final Test Accuracy: 96.32% 
